In [1]:
# Downgrade protobuf version due 
# !pip install protobuf==3.20.0
# !pip list
# pip install transformers[torch]==4.3

In [11]:
# Load libraries 
import os
import json
import rouge
import pandas as pd
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration

In [4]:
# Load pretrained model
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")
model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

Downloading:   0%|          | 0.00/793k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/657 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [7]:
# Check current file
os.listdir(
"dataset/liputan6_data/liputan6_data/canonical/train"
)[:5]

['100000.json', '100002.json', '100003.json', '100004.json', '100005.json']

In [9]:
# Load sample json data
data_sample = json.load(open(
    "dataset/liputan6_data/liputan6_data/canonical/train/100000.json", "r"
))

In [22]:
# Define sample preprocessed article 
ARTICLE_TO_SUMMARIZE = " ".join(sum(data_sample['clean_article'], []))
print(ARTICLE_TO_SUMMARIZE)

Liputan6 . com , Jakarta : Presiden Susilo Bambang Yudhoyono menekankan bahwa tantangan terbesar yang dihadapi bangsa-bangsa Asia dan Afrika saat ini adalah masalah kemiskinan yang sangat buruk . Yudhoyono berharap masalah ini menjadi pembahasan penting dalam Konferensi Tingkat Tinggi Asia-Afrika . Demikian pidato Yudhoyono saat membuka KTT Asia-Afrika di Jakarta Convention Centre , Jakarta , Jumat ( 22/4 ) [ baca : Presiden Yudhoyono Resmi Membuka KAA ] . Pada awal pidatonya , Yudhoyono para peserta untuk mengheningkan cipta sejenak bagi korban gempa dan Tsunami di Nanggroe Aceh Darussalam dan Nias , Sumatra Utara . Selanjutnya , Yudhoyono menegaskan pertemuan ini bukanlah sekadar romantisme , tetapi upaya bersama untuk memecahkan masalah yang dihadapi kedua benua . Acara pembukaan konferensi ini juga dihadiri Sekretaris Jenderal Perserikatan Bangsa-Bangsa Kofi Annan yang datang sebagai tamu kehormatan . Meski fokusnya adalah mempererat hubungan antara Asia dan Afrika , konferensi ini

In [23]:
# Encode sentence using pretrained model
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors = "pt")

In [31]:
%%time
# Generate summary
summary_ids = model.generate(
    input_ids, min_length = 25, max_length = 75, 
    num_beams = 10, repetition_penalty = 2.5, 
    length_penalty = 1.0, early_stopping = True, 
    no_repeat_ngram_size = 3, use_cache = True, 
    temperature = 0.9, top_k = 10, top_p = 0.9
)

CPU times: total: 26.2 s
Wall time: 11.3 s


In [32]:
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

Presiden Yudhoyono berharap masalah kemiskinan menjadi pembahasan penting dalam KTT Asia-Afrika. Acara pembukaan konferensi ini juga dihadiri Sekjen PBB Kofi Annan yang datang sebagai tamu kehormatan.


![](https://miro.medium.com/v2/resize:fit:1400/1*g-Ucb9QosEBkRI0Jv31ZPQ.png)

In [34]:
# Inherit ROUGE scoring
rouge_scoring = rouge.Rouge()

In [35]:
rouge_scoring.get_scores(
    ARTICLE_TO_SUMMARIZE, summary_text
)

[{'rouge-1': {'r': 0.9230769230769231,
   'p': 0.11822660098522167,
   'f': 0.2096069848866345},
  'rouge-2': {'r': 0.76, 'p': 0.06418918918918919, 'f': 0.1183800608689745},
  'rouge-l': {'r': 0.9230769230769231,
   'p': 0.11822660098522167,
   'f': 0.2096069848866345}}]